In [1]:
import torch
import random
from collections import deque
from torch.nn.functional import relu, prelu

# Graph search

In [2]:
#todo: collect all incoming edges for coalescence (search how they do it so it's consistent)
#todo: after implementing message pass algorithm, move it to forward pass (if possible... and if it doesn't overcomplicate modularly changing out attention structures...)

embedding_size = 5
class GraphSearch:
    def __init__(self, graph, protocol='DFS'):
        self.graph = graph
        self.protocol = protocol
        self.visited = set() #<1 remove or adjust depending on walk style in message passing alg...
    
    def step(self, buffer, coalesce_func=lambda x:x):
        if not buffer:
            return None, buffer
        
        if self.protocol == 'BFS':
            current_id = buffer.popleft()
        else:
            current_id = buffer.pop()
        print(current_id)
        if current_id in self.visited: #<1
            return None, buffer

        self.visited.add(current_id)
        current = self.graph[current_id]
        embedding, neighbours = current
        embedding = coalesce(embedding)
        print(f"Visiting node: {current_id}")
        
        for neighbour in neighbours:
            if neighbour not in self.visited and neighbour not in buffer:
                buffer.append(neighbour)
                neighbour_embedding, _ = self.graph[neighbour]
                print(f"Added node {neighbour} to buffer.")
        
        return current, buffer

graph = {
    1: [torch.rand(embedding_size), (2, 3)],
    2: [torch.rand(embedding_size), (4, 5)],
    3: [torch.rand(embedding_size), (6,)],
    4: [torch.rand(embedding_size), (7,)],
    5: [torch.rand(embedding_size), (8, 9)],
    6: [torch.rand(embedding_size), (10,)],
    7: [torch.rand(embedding_size), (1,)],
    8: [torch.rand(embedding_size), (6, 10)],
    9: [torch.rand(embedding_size), (7,)],
    10: [torch.rand(embedding_size), (5,)]
}

edge_1 = torch.tensor([1.9,2.7,3.5],dtype=torch.float16)
edge_2 = torch.tensor([9.1,10.3,12.5],dtype=torch.float16)
edge_3 = torch.tensor([9.1,10.3,12.5],dtype=torch.float16)




### cashew: try graphgym package :)
# Proposed GNN architecture:
## Transformation block >>>
### * linear
### * batch norm * <  my intuition is that this could replace mean() operation, just use vectorized sum() to reduce computational complexity.
### * dropout * < On linear layer in the message function
### * activation * < parametric relu = max(x,0) + alpha * min(x,0) ... alpha is trainable.
### * attention * < I'm not sure if relational weights as in RGCN fall into this category. If they are complementary in any way.
## <<< End transformation block
### * aggregation by some problem dependant function i.e. mean(), min/max/avg..._pooling(), lstm(cat(edge_embeddings)) ...
#### aggregation note: inverted degree matrix * adjacency matrix = avg(adjacency matrix)

# classical GCN:

## important design choice here... use batch norm after each layer? Normalize explicitly?
## messages = layer weight * normalized messages from prev layers
## aggregation = sum(messages) --> relu

### todo: add weighted average method. I.e. learnable row vector of size feature dim (must vectorize torch.mean explicitly for this)

In [3]:
# def aggregate(features): #this creates an aggregation directly as a matrix op
#     agg = torch.stack(features)
#     return agg, torch.mean(agg, dim=0)

# def coalesce(features):

# def normalize(messages):
#     num = len(messages)
    

# def relu(features):
#     return relu(features)

# GraphSAGE
## aggregate incoming messages (can be mean(messages, dim=0), can be a max pooling on mlp(message), can be LSTM(shuffle(messages) as mini-batch), can be sum without average (maybe this leads to batch norm later?)
## concat current node message --> relu --> send

## Uses L2 Norm as root squared error of embeddings at every layer.

# GAT

# Architectural notes:

## GCN, but vector weighting matrix is learned (which nodes to attend to and ./ unsure./ which parts of the embedding vectors/features to attend to \.\.

### how can we handle permutation invariance?

### seems that attention weights are graph conditional on search algorithm dependant

### ^ wrong. It's a function of (and on) embeddings of different node embeddings at the previous step.

### softmax ()

### parameter matrix a can be a parameter matrix on a learned single layer mlp that processes the concatenated input vectors.

### parameter matrix a is learned together with weight matrix w.

### multi-head attention, multiple relu(a) matrices.

### each a is initialized randomly, then aggregated to produce a single output

### can be parallellized worker per message.

### sparse matrix... fixed number of parameters.

# Implications/discussion

### asymmetric importance weighting

### weights are still independant of graph size, even though more complex analysis of the graph can be performed.

### graph attention mechanism scales linearly in graph size due to locality

### cool visualization of attention mechanism (implicit clustering) cora citation paper

### improved performance over GCN in some cases.




# Test stuff and transform into main() below...

In [4]:
coalesce((edge_1,edge_2, edge_3))

NameError: name 'coalesce' is not defined

In [ ]:
dfs = GraphSearch(graph, protocol='BFS')

walk = True
lambda_ = 0.1
node_id = 1
stack = deque([node_id])
print(stack)
while walk:
    node_id, stack = dfs.step(stack)
    if random.random() < lambda_:
        walk = False
print(graph)